# Beehive monitoring

Ronald Tabernig, William Albert, Hannah Weiser, July 2025 

This notebook demonstrates the preparation of a beehive monitoring example, including the bees detection workflow. The dataset contains **6 scans** captured each 40 seconds on 14 may 2025.

## Imports

In [1]:
import py4dgeo
import vapc
import laspy
import numpy as np
import os
import sys
import json
sys.path.insert(0, "../src")
from fourdgeo import projection
from fourdgeo import utilities
from fourdgeo import change

# File import and handling
from pathlib import Path
import zipfile, urllib.request, shutil
from glob import glob

# Image handling
from PIL import Image, ImageDraw, ImageFont
from IPython.display import HTML

## Get the data

In [4]:
# Handle file download/reading here
data_url = "https://heibox.uni-heidelberg.de/f/6c2a4e6755b74d1abad0/?dl=1"
file_name = "beehive.zip"
data_folder = "data/beehive"

if not Path(data_folder).exists():
    with urllib.request.urlopen(data_url) as response, open(file_name, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        zip_ref.extractall(data_folder)
    
    # Deleting the .zip file
    os.remove(file_name)

## Bee detection

### Isolate the bees

In [ ]:
observations = {"observations": []}

# Gather & sort only the .laz files
laz_paths = list(Path(data_folder).glob("*.laz"))
laz_paths = sorted(laz_paths)

voxel_size = 0.05

dh_mask = vapc.DataHandler(os.path.join(data_folder, "mask.laz"))
dh_mask.load_las_files()

coords = dh_mask.df[["X", "Y", "Z"]].to_numpy()
distances = np.linalg.norm(coords, axis=1)
dh_mask.df = dh_mask.df[distances <= 12].reset_index(drop=True)

vp_mask = vapc.Vapc(voxel_size=voxel_size, return_at="center_of_voxel")
vp_mask.get_data_from_data_handler(dh_mask)
# vp_mask.compute_voxel_buffer(2)
#vp_mask.reduce_to_voxels()
vp_mask.voxel_index = False

for enum, laz_path in enumerate(laz_paths):
    dh_epoch = vapc.DataHandler(laz_path)
    dh_epoch.load_las_files()

    coords = dh_epoch.df[["X", "Y", "Z"]].to_numpy()
    distances = np.linalg.norm(coords, axis=1)
    dh_epoch.df = dh_epoch.df[distances <= 12].reset_index(drop=True)

    vp_epoch = vapc.Vapc(voxel_size=voxel_size, return_at="center_of_voxel")
    vp_epoch.get_data_from_data_handler(dh_epoch)
    # vp_epoch.reduce_to_voxels()
    vp_epoch.select_by_mask(vp_mask, segment_in_or_out="out")
    dh_epoch.df = vp_epoch.df

    masked_pc_path = os.path.join(f"out/beehive/masked_{enum:02d}_vox_{int(voxel_size*1000):03d}.laz")
    dh_epoch.save_as_las(masked_pc_path)

### Cluster the bees

## Projections
### Prepare the configuration file

We use a configuration dictionary that contains general project settings like the `output_folder` and the relevant settings for the point cloud projection. For the projection, parameters like the `camera_position` and the `resolution_cm` are essential.

In [ ]:
configuration = {
    "project_setting": {
        "project_name": "Beehive",
        "output_folder": "./out",
        "temporal_format": "%y%m%d_%H%M%S",
        "silent_mode": True,
        "include_timestamp": False
    },
    "pc_projection": {
        "pc_path": "",
        "make_range_image": True,
        "make_color_image": False,
        "top_view": False,
        "save_rot_pc": False,
        "resolution_cm": 12.5,
        "camera_position": [
            0.0,
            0.0,
            0.0
        ],
        "rgb_light_intensity": 100,
        "range_light_intensity": 10,
        "epsg": None
    }
}

### Generate the background image

We now generate the background images. For this, we are using classes and functions from the `fourdgeo` library. The class `PCloudProjection` directly takes our configuration file as input and writes them into our specified `output_folder`.

### Project the change events onto the image background

Here, we also project the change events onto the background image using the `ProjectChange` class. The `observation` GeoJSON files are written to the `output_folder`.

### Display the bee clusters in the site

We generate a GIF of the time series and display the bee clusters using the points.